# Part I: Linearization

The first step of designing the LQR controller for our quadrotor is to linearize the system about the hover state. The equations for the 3D quadrotor, however, are very complicated, and computing the necessary derivatives is too tedious to want to do by hand. Therefore, we will use a computer algebra system (CAS) to save us a lot of time in computing derivatives. Specifically, we will be using the [SymPy](https://docs.sympy.org/latest/index.html) package to help us along the way. If you have experience with Mathematica or MATLAB's Symbolic Toolkit, SymPy offers many of the same features but in a Python interface, and even allows us to convert the symbolic functions that we will derive into efficient numerical ones.

To get started, we will demonstrate how to use SymPy to symbolically linearize the planar quadrotor model you worked with by hand in the previous assignment, then leave it to you to linearize the 3D model. We also suggest reading the brief tutorial given in the SymPy documentation, which will cover most of what you will need for this assignment.

We begin by importing the functions we need from SymPy.

In [1]:
import sympy as sp
import numpy as np
from sympy.physics.vector import dynamicsymbols as dynamicsymbols

Next, we define the symbolic variables we need to describe the equations of motion. The function ``dynamicsymbols`` creates symbols that vary in time, i.e. ``dynamicsymbols('x')`` will create a symbol $x(t)$ as opposed to $x$.

In [2]:
m, g, I, r, t = sp.symbols('m g I r t')
u1, u2  = sp.symbols('u1 u2')
x, y, theta = dynamicsymbols('x y theta')

We also define some variables as shorthand for the time derivatives of our state variables.

In [3]:
x_dot = sp.diff(x, t)
y_dot = sp.diff(y, t)
theta_dot = sp.diff(theta, t)

Now, we write out the equations of motion. Note that `sp.Matrix` is used to create both matrices and vectors in a manner similar to `np.array`.

In [4]:
state = sp.Matrix([x, y, theta, x_dot, y_dot, theta_dot])
input = sp.Matrix([u1, u2])

dynamics = sp.Matrix([x_dot, y_dot, theta_dot,
                      -u1 / m * sp.sin(theta),
                      u1 / m * sp.cos(theta) - g,
                      u2 / I])

Finally, we differentiate and plug in numerical values via the `subs` method. In this case, the hover state is chosen to be the $0$ vector and the hover input is $u_1 = m g$. We'll leave the parameters of the system as symbolic values for now so we can see the structure of the result.

In [5]:
A = dynamics.jacobian(state)
B = dynamics.jacobian(input)

In [6]:
A.subs([(u1, m * g), (theta, 0)])

Matrix([
[0, 0,  0, 1, 0, 0],
[0, 0,  0, 0, 1, 0],
[0, 0,  0, 0, 0, 1],
[0, 0, -g, 0, 0, 0],
[0, 0,  0, 0, 0, 0],
[0, 0,  0, 0, 0, 0]])

In [7]:
B.subs([(theta, 0)])

Matrix([
[  0,   0],
[  0,   0],
[  0,   0],
[  0,   0],
[1/m,   0],
[  0, 1/I]])

The last thing we need to discuss is turning the A and B matrices in the previous cells into `np.ndarray` types. The reason we want to do this is that after we replace the remaining system parameters with numerical values, SymPy will represent these values in a way that is very precise, but incredibly inefficient for the numerical work we need to do. Instead, we want to work with NumPy's numerical arrays, i.e. `np.ndarray`. These cannot handle symbolic values like SymPy's matrices and will not operate at same level of precision, but they are much more efficient for computational purposes.

In [8]:
A_arr = np.array(A.subs([(u1, m * g), (theta, 0), (m, 0.03), (g, 9.81)])).astype(float)

B_arr = np.array(B.subs([(theta, 0), (m, 0.03), (I, 1.419e-5)])).astype(float)

In each of these two lines, we are doing the following. First, we substitute in all the hover state and parameter numerical values as before. Next, we call `np.array` to convert from `sp.Matrix` to `np.ndarray`. However, we still have to tell numpy which data type to use to represent the numerical values in this matrix. To do so, we call the method `np.ndarray.astype()` with the type we want, `float`. You can see these matrices have a different numerical representation now:

In [9]:
print(A_arr)
print()
print(B_arr)

[[ 0.    0.    0.    1.    0.    0.  ]
 [ 0.    0.    0.    0.    1.    0.  ]
 [ 0.    0.    0.    0.    0.    1.  ]
 [ 0.    0.   -9.81  0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.  ]]

[[0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [3.33333333e+01 0.00000000e+00]
 [0.00000000e+00 7.04721635e+04]]


## Problem 1
Now it's your turn! In the following function, please write the code to linearize the 3D quadrotor model that was discussed in class. The function should return A and B matrices computed like above as a tuple of `np.ndarray` structures. In addition to the hover state and input, you should substitute the following values for the physical parameters of the system:

- $m = 0.03kg$
- $g = 9.81 \frac{m}{s^2}$
- $I_{xx} = 1.4194e-05$
- $I_{yy} = 1.4089e-05$
- $I_{zz} = 2.9741e-05$

Note the full state of the system $\mathbf{x}$ and system inputs $\mathbf{u}$ are the variables, in order:

$$\begin{align}\mathbf{x} = \begin{bmatrix}x\\ y\\ z\\ \phi\\ \theta\\ \psi\\ \dot{x}\\ \dot{y}\\ \dot{z}\\ p\\ q\\ r\end{bmatrix} & & \mathbf{u} = \begin{bmatrix}F_{tot}\\ M_1\\ M_2\\ M_3\end{bmatrix}\end{align}$$

In [10]:
def linearize_quadrotor3d() -> (np.ndarray, np.ndarray):
    return (A_arr, B_arr)

# Part II: LQR Control

In this part, we are going to use the linearization function you wrote to design the LQR controller. First, we are going to import the various classes and functions we need.

In [ ]:
from mae345 import plotting, Crazyflie, animate_quad
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as la
from IPython.display import HTML

## Problem 2

Now, in the cell below, you will fillout the function that will actually solve the LQR problem. That is, given the matrices $A, B, Q, R$, you should compute the $K$ that minimizes the LQR cost function. You are encouraged to use any function in the `scipy.linalg` library. Also, we use the convention that the stabilizing control input is given by $\mathbf{u} = \mathbf{K}\mathbf{x}$ (as opposed to $\mathbf{u} = -\mathbf{K}\mathbf{x}$).

In [11]:
def lqr(A: np.ndarray, B: np.ndarray, Q: np.ndarray, R: np.ndarray) -> np.ndarray:    
    P = la.solve_continuous_are(A, B, Q, R)
    K = -np.linalg.inv(R) @ (B.T @ P)
    return K

## Problem 3

Finally, you will use all the code you have written thus far to actually stabilize your Crazyflie. The following code implements a version of the Crazyflie class from our `mae345` library that uses the linearization and LQR functions you just wrote. This class is useful as it provides simulation and animation functionality to varify your controller is working. It also saves out $\mathbf{K}$ so it can be loaded onto the actual quadrotor for control. You will need to adjust the gains yourself, but we filled in a few for you to get started.

Once you have a set of gains you want to try on the quadrotor, simply navigate to the `~/MAE345` in your terminal and run the command `sudo python3 run_lab1.py`. It will load the feedback matrix corresponding to your LQR controller and run it on the drone. The drone takes off and lands using the Crazyflie's own PID controller, but your LQR controller will stabilize the drone once it is hover a half meter off the ground. The drone beeps when switching controllers. You must submit the following to confirm that you got the drone to hover successfully:
- A video of your drone hovering
- The file `quad_data/quad_traj.npz`
- This notebook file

In [ ]:
class CrazyflieLQR(Crazyflie):
    def __init__(self, Q: np.ndarray, R: np.ndarray, hover_pos: np.ndarray):
        super().__init__()
        self._hover_pos = hover_pos
        self._hover_state = np.concatenate([self._hover_pos, np.zeros(9)])
        (A, B) =  linearize_quadrotor3d()
        self._K = lqr(A, B, Q, R)
        
        print('Using K matrix:')
        print(self._K)
        
        np.save('quad_data/lqr_gains', self._K)
    
    def controller(self, state: np.ndarray, t: float) -> np.ndarray:
        return self._K @ (state - self._hover_state) + np.array([self.mass * self.gravity, 0, 0, 0])

cf = CrazyflieLQR(np.diag([1000, 1000, 2000, 0.001, 0.001, 1, 100, 100, 1, 0.005, 0.005, 1]), 2 * np.diag([1e5, 6e9, 6e9, 1e4]), np.zeros(3))

This next cell simulates the quadrotor flying with a random initial condition for five seconds and plots some of the states. You should see the system stabilize, or your gains will almost certainly not work on the real Crazyflie. Feel free to change the plotted variables.

In [ ]:
# Uncomment the following line if you want to use the same random initial condition.
# np.random.seed(0)

ic = np.random.rand(12) * 1.2
ic[0:3] = ic[0:3] / 5
ic[6:9] = ic[6:9] / 5

times, states, inputs = cf.simulate(ic, 5, 0.01, clip_input=False)

plt.rcParams["figure.figsize"] = (16,8)
fig = plt.figure()
ax = fig.add_subplot(121, title='Position')
ax.plot(times, states[0, :], label='x')
ax.plot(times, states[1, :], label='y')
ax.plot(times, states[2, :], label='z')
ax.legend()

ax = fig.add_subplot(122, title='Angles')
ax.plot(times, states[3, :], label='phi')
ax.plot(times, states[4, :], label='theta')
ax.plot(times, states[5, :], label='psi')
ax.legend()

This cell animates the simulation data computed / plotted in the previous cell.

In [ ]:
HTML(animate_quad(0.1, states).to_html5_video())

Finally, this cell plots the result of your physical experiments, i.e. the data logged by the most recent run of `run_lab1.py` in `quad_data/quad_traj.npz`.